In [237]:
from os import path

In [252]:
import os

In [67]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
%matplotlib inline

In [71]:
from skimage.transform import rescale
from skimage import io

In [192]:
import keras
import keras.layers as L
from keras.applications.resnet50 import ResNet50

In [364]:
def adjust_size(img, size=224):
    scaled_img = rescale(img, size/max(img.shape))
    if len(scaled_img.shape) == 2:
        scaled_img = np.stack([scaled_img]*3).transpose(1, 2, 0)
    for i in [0, 1]:
        if scaled_img.shape[i] < size:
            result = np.zeros([size, size, 3])
            gap = (size - scaled_img.shape[i]) // 2
            subslice = [slice(gap, gap + scaled_img.shape[i]), slice(None)]
            result[subslice[i], subslice[1 - i]] = scaled_img
            return result
    return scaled_img

def load_images(path, names, size=224):
    result = np.empty([len(names), size, size, 3])
    for i, name in enumerate(names):
        result[i, :, :, :] = adjust_size(io.imread(os.path.join(path, 'images', name)), size=size)
    return result
                                         
                                         
def load_targets(path, index):
    df = pd.read_csv(os.join(path, 'gt.csv'), index_col='filename')
    return np.array(df.iloc[index, 1])                                 

                                         
def iter_batches(path, target=False, batch_size=100, shuffle=True, cycle=True):
    files = np.array(os.listdir(os.path.join(path, 'images')))[:100]
    if target:
        targets = pd.read_csv(os.path.join(path, 'gt.csv'))
    while True:
        if shuffle:
            np.random.shuffle(files)
        for start in range(0, len(files), batch_size):
            names = files[start: start + batch_size]
            images = load_images(path, names)
            if target is not None:
                yield images, np.array(df.loc[names]).T[0]
            else:
                yield images
                
        if not cycle: break

In [ ]:
resnet = ResNet50()

In [366]:
output = L.Dense(50, activation='softmax', name='dense')(resnet.get_layer(index=-2).output)
model = keras.models.Model(input=resnet.input, output=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [367]:
img = io.imread('data/00_input/train/images/{:04d}.jpg'.format(0))

In [ ]:
model.fit_generator(iter_batches('data/00_input/train', target=True), steps_per_epoch=10)

Epoch 1/1


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [313]:
for i in range(2500):
    img = io.imread('data/00_input/train/images/{:04d}.jpg'.format(i))
    if len(img.shape) != 3:
        img = np.stack(img, img, img)
        plt.imshow(img)


TypeError: only size-1 arrays can be converted to Python scalars

In [318]:
A = np.arange(25).reshape(5, 5)

In [322]:
B = 

In [325]:
B.shape

(5, 5, 3)

In [329]:
img.shape

(335, 500, 3)